In [2]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATASETS
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE CELL.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote
from urllib.error import HTTPError
from zipfile import ZipFile

CHUNK_SIZE = 40960
DATASET_MAPPING = 'h-and-m-personalized-fashion-recommendations:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F31254%2F3103714%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20220324%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20220324T091850Z%26X-Goog-Expires%3D259199%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1b88b39ef0ff88010f48880a34acc72d51ea6d6c56b4659e530d07855dd8743271ac221d27cab8787c324d2157c2222ee59771145f57bbfc798ecc770b5c77eab04aae9c15b6c65237add59adfa6c1b20bf697562afd5f66d4b7e5907e327d9b2b0197404e0c3cb8c2338de36e129839ec2c84c247bcff9dd7bf3dfb42f2abdba1b7bc40043acf9aa56c4065ede417797fbb319ab3fd1c744aefa93245de175a87863c67cdef511c499e5c8f09de480e01ecadab3386b2d84905e312eee1e60314e1acd599c760669aa297af5436846360e2f27bc6968764f21fa4ad9552c5c76d3e07dc408c8668fcfcf46c6f569eef92b444c3147a708efc122217aad09e20'
KAGGLE_INPUT_PATH='/home/kaggle/input'
KAGGLE_INPUT_SYMLINK='/kaggle'

os.makedirs(KAGGLE_INPUT_PATH, 777)
os.symlink(KAGGLE_INPUT_PATH, os.path.join('..', 'input'), target_is_directory=True)
os.makedirs(KAGGLE_INPUT_SYMLINK)
os.symlink(KAGGLE_INPUT_PATH, os.path.join(KAGGLE_INPUT_SYMLINK, 'input'), target_is_directory=True)

for dataset_mapping in DATASET_MAPPING.split(','):
    directory, download_url_encoded = dataset_mapping.split(':')
    download_url = unquote(download_url_encoded)
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as zipfileres, NamedTemporaryFile() as tfile:
            total_length = zipfileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes zipped')
            dl = 0
            data = zipfileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = zipfileres.read(CHUNK_SIZE)
            print(f'\nUnzipping {directory}')
            with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue
print('Dataset import complete.')


FileExistsError: [Errno 17] File exists: '/home/kaggle/input'

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from scipy.spatial.distance import euclidean, cityblock, cosine

In [5]:
trans = pd.read_csv('/home/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
articles=pd.read_csv('/home/kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv')

In [6]:
trans1=trans.drop_duplicates()

In [7]:
trans1=trans1.groupby('customer_id').tail(1)

In [8]:
trans2=trans1[(trans['t_dat'] > "2020-06-23") & (trans['t_dat'] < "2020-09-23")]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [9]:
trans2.shape

(525075, 5)

In [37]:
df=trans2[["customer_id","article_id"]].copy()
feature=articles[['article_id','product_group_name', 
       'graphical_appearance_name',
       'perceived_colour_value_name',
       'perceived_colour_master_name','section_name',
       'garment_group_name']]


In [38]:
df1 = df.merge(feature, on='article_id')
df1.head()

,customer_id,article_id,product_group_name,graphical_appearance_name,perceived_colour_value_name,perceived_colour_master_name,section_name,garment_group_name
0,0021b5b5459c57173a0aee5e04276d3fa2cbd969c19c52cbd2461f4ef8fae909,913030001,Swimwear,Solid,Dark,Black,"Womens Swimwear, beachwear",Swimwear
1,3a210003afdadbcfad20ef9fbd03158b360e1adf2e2b7cd2a1389c79b17e35d7,913030001,Swimwear,Solid,Dark,Black,"Womens Swimwear, beachwear",Swimwear
2,693f940065f20fedf517feee4da366a6c46b496b6a11ca880d93d1675bf03e27,913030001,Swimwear,Solid,Dark,Black,"Womens Swimwear, beachwear",Swimwear
3,01c1529d0cac7f12b6e327e9f8e4c2b3b371a4f1d8fb61932f1fbeb328cb62c1,913030001,Swimwear,Solid,Dark,Black,"Womens Swimwear, beachwear",Swimwear
4,025ed6f246d5ffd4b41915622062818ea0bafdb64f417ed20c25a841b1cec816,913030001,Swimwear,Solid,Dark,Black,"Womens Swimwear, beachwear",Swimwear


In [12]:
pd.set_option('display.max_colwidth', None)
df1.info(20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 525075 entries, 0 to 525074
Data columns (total 5 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   customer_id                   525075 non-null  object
 1   article_id                    525075 non-null  int64 
 2   product_group_name            525075 non-null  object
 3   graphical_appearance_name     525075 non-null  object
 4   perceived_colour_master_name  525075 non-null  object
dtypes: int64(1), object(4)
memory usage: 24.0+ MB


In [39]:
featurelist=['product_group_name', 'graphical_appearance_name','perceived_colour_value_name','perceived_colour_master_name', 'section_name','garment_group_name']
dummies_df = pd.get_dummies(df1, columns=featurelist)

In [14]:
#trans.groupby('customer_id').count()

In [40]:
item_feature = dummies_df.copy()
item_feature=item_feature.drop_duplicates(subset='article_id')
item_feature = item_feature.drop('customer_id', axis=1)
item_feature = item_feature.set_index('article_id')
#item_feature.to_csv('item_feature.csv',index=True)
#item_feature.info(verbose=True, null_counts=True)
item_feature


,product_group_name_Accessories,product_group_name_Bags,product_group_name_Cosmetic,product_group_name_Furniture,product_group_name_Garment Full body,product_group_name_Garment Lower body,product_group_name_Garment Upper body,product_group_name_Garment and Shoe care,product_group_name_Items,product_group_name_Nightwear,...,garment_group_name_Shorts,garment_group_name_Skirts,garment_group_name_Socks and Tights,garment_group_name_Special Offers,garment_group_name_Swimwear,garment_group_name_Trousers,garment_group_name_Trousers Denim,"garment_group_name_Under-, Nightwear",garment_group_name_Unknown,garment_group_name_Woven/Jersey/Knitted mix Baby
article_id,,,,,,,,,,,,,,,,,,,,,
913030001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
808748001,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
688728003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
684209019,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
843873001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903910002,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
795594013,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
772659001,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [41]:
articleid_list=item_feature.index.tolist()

In [42]:
len(articleid_list)

27748

In [43]:
contentbased=trans2.copy()
#contentbased.drop(['t_dat','price','sales_channel_id'], axis = 1, inplace = True)
#contentbased.set_index('customer_id', inplace=True)
#contentbased.to_csv('customers_3months.csv',index=False)
contentbased

,t_dat,customer_id,article_id,price,sales_channel_id
27883973,2020-06-24,0021b5b5459c57173a0aee5e04276d3fa2cbd969c19c52cbd2461f4ef8fae909,913030001,0.022864,2
27883984,2020-06-24,002a9d5d9db272d51e99bc7e41e80a2b68720be75090831f3983174d290b7067,808748001,0.022881,1
27883985,2020-06-24,002c461a5ac24e9b3d89c3af593e3bcea34453cea40e815000b84039a455b4c1,688728003,0.025407,2
27884017,2020-06-24,005a2df6006a9b28dc6c33f1f994902d7bbed7251f80bbde30ed134950c0284b,684209019,0.015458,2
27884022,2020-06-24,005cd5673d04af025b8c7a4d4c8bcaa879da7478bccb88541191ec0f9586ab7c,843873001,0.011000,1
...,...,...,...,...,...
31788318,2020-09-22,ffd4cf2217de4a0a3f9f610cdec334c803692a18af08ac55b4c6fb955cc836b1,856440002,0.042356,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f99982410630ac51314356,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e6936f26ea7ecaa68a1,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e593881ae6007d775f0f,833459002,0.006763,1


In [44]:
contentbased['article_id'].nunique()

27748

In [85]:
customer_id="005cd5673d04af025b8c7a4d4c8bcaa879da7478bccb88541191ec0f9586ab7c"
#article_id=contentbased[contentbased['customer_id']==customer_id]['article_id'].values[0]
article_id=contentbased.loc[contentbased['customer_id'] == customer_id, 'article_id'].iloc[0]
article_id

843873001

In [71]:
def recommend(article_id):
    # create dataframe used to store distances between recipes
    item_distance = pd.DataFrame(data=item_feature.index)
    
    # remove rows where index is equal to the inputted recipe_id
    item_distance = item_distance[item_feature.index != article_id]
    
    # add a distance column that states the inputted recipe's distance with every other recipe
    item_distance['distance'] = item_distance["article_id"].apply(lambda x: euclidean(item_feature.loc[x],item_feature.loc[article_id]))
    
    # sort the allRecipes by distance and take N closes number of rows to put in the TopNRecommendation as the recommendations
    item_distance.sort_values(by='distance',inplace=True)
    
    # for each recipe in TopNRecommendation, input to defined lists
    
    # return dataframe with the inputted recipe and the recommended recipe's normalized nutritional value
    return(item_distance['article_id'].head(10).tolist())

In [86]:
rcmnds=recommend(article_id)
rcmnds

[843872001,
 723349002,
 721990001,
 739590040,
 687034024,
 739590041,
 715343007,
 833486002,
 725661001,
 739590024]

In [87]:
#prediction=[]
#for x in range(len(articleid_list[0:200])):
    #rmd=recommend(articleid_list[x])
    #prediction.append(rmd)

In [88]:
#print(prediction)

In [89]:
def plot_rcmnd(rcmnds):
    fig = plt.figure(figsize=(20, 10))
    for item, i in zip(rcmnds, range(1, 11)):
        item = '0' + str(item)
        path = "/home/kaggle/input/h-and-m-personalized-fashion-recommendations/images"
        sub = item[:3]
        image = path + "/"+ sub + "/"+ item +".jpg"
        image = plt.imread(image)
        fig.add_subplot(1, 10, i)
        plt.imshow(image)

In [90]:
plot_rcmnd(rcmnds)

FileNotFoundError: [Errno 2] No such file or directory: '/home/kaggle/input/h-and-m-personalized-fashion-recommendations/images/084/0843872001.jpg'

<Figure size 1440x720 with 0 Axes>

In [78]:
def plot_prev(article_id):
    fig = plt.figure(figsize=(20, 10))
    path = "/home/kaggle/input/h-and-m-personalized-fashion-recommendations/images"
    item = '0' + str(article_id)
    sub = item[:3]
    image = path + "/"+ sub + "/"+ item +".jpg"
    image = plt.imread(image)
    fig.add_subplot(1, 6,1)
    plt.imshow(image)

In [80]:
plot_prev(408875001)

FileNotFoundError: [Errno 2] No such file or directory: '/home/kaggle/input/h-and-m-personalized-fashion-recommendations/images/040/0408875001.jpg'

<Figure size 1440x720 with 0 Axes>

In [51]:
#k=4
#def plot_rcmnd(recommend):
    #fig = plt.figure(figsize=(20, 10))
    #path = "/home/kaggle/input/h-and-m-personalized-fashion-recommendations/images"
    #for item in recommend:
        #item = '0' + str(item)
        #sub = item[:3]
        #image = path + "/"+ sub + "/"+ item +".jpg"
        #image = plt.imread(image)
        #fig.add_subplot(1, 6, i)
        #plt.imshow(image)

In [ ]:
#plot_rcmnd(recommend)

In [ ]:
#plot_prev(prev_items)

In [ ]:
#plot_rcmnd(recommend)

In [91]:
productname = pd.read_csv('/home/kaggle/input/h-and-m-personalized-fashion-recommendations/productname_list.csv')

In [92]:
productname

,article_id,prod_name
0,108775015,Strap top
1,108775044,Strap top
2,108775051,Strap top (1)
3,110065001,OP T-shirt (Idro)
4,110065002,OP T-shirt (Idro)
...,...,...
105537,953450001,5pk regular Placement1
105538,953763001,SPORT Malaga tank
105539,956217002,Cartwheel dress
105540,957375001,CLAIRE HAIR CLAW
